In [46]:
%pip install radcad
%pip install pandas
%pip install numpy
%pip install plotly

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [47]:
from radcad import Model, Simulation, Experiment
import math
import pandas as pd
import plotly
from numpy import random
from functools import partial

def _update_from_signal(
    state_variable,
    signal_key,
    params,
    substep,
    state_history,
    previous_state,
    policy_input,
):
    return state_variable, policy_input[signal_key]


def update_from_signal(state_variable, signal_key=None):
    """A generic State Update Function to update a State Variable directly from a Policy Signal
    Args:
        state_variable (str): State Variable key
        signal_key (str, optional): Policy Signal key. Defaults to None.
    Returns:
        Callable: A generic State Update Function
    """
    if not signal_key:
        signal_key = state_variable
    return partial(_update_from_signal, state_variable, signal_key)


random.seed(1234)

initial_state = {
    'rune_vault': 0.0,
    'asset_vault': 0.0,
    'rune_pool': 400.0,
    'asset_pool': 1.0,
    'rune_stake': 0.0,
    'asset_stake': 0.0,
    # How does vault stake work? 
    'asset_stakers': [{'asset':10, 'asset_lp_ratio': 0, 'asset_staked': 0} for i in range(10)],
    'rune_stakers': [{'rune':10000, 'rune_lp_ratio': 0, 'rune_staked': 0} for i in range(10)],
}

params = {
    # price in USD to calculate value of gas fee.
    'asset_price': [2000.0],
    'rune_price': [5.0],
    'thor_chain_tx_fees': [0.2],
    'asset_chain_tx_fees': [0.01],
    'lp_threshold': [1000.0],
    'max_swap_amount': [50.0],
    'asset_stake_amount': [1500],
    'rune_stake_amount': [800]
}

In [48]:
def to_rune(val_usd, params):
    return val_usd / params['rune_price']

def to_asset(val_usd, params):
    return val_usd / params['asset_price']

def from_rune(val_rune, params):
    # returns USD value
    return val_rune * params['rune_price']

def from_asset(val_asset, params):
    # returns USD value
    return val_asset * params['asset_price']

def swap_to_pool(swap_amount_from, from_pool, to_pool):
    return ((swap_amount_from * from_pool * to_pool) / ((swap_amount_from + from_pool) ** 2))

def swap_to_rune(swap_amount_asset, previous_state):
    return swap_to_pool(swap_amount_asset, previous_state['asset_pool'], previous_state['rune_pool'])

def swap_to_asset(swap_amount_rune, previous_state):
    return swap_to_pool(swap_amount_rune, previous_state['rune_pool'], previous_state['asset_pool'])

def lp_fees(swap_amount_from, from_pool, to_pool, to_tx_fees):
    return to_tx_fees + ((swap_amount_from**2)*to_pool/(swap_amount_from+from_pool)**2)
    
def lp_fees_to_asset(swap_amount_rune, params, previous_state):
    return lp_fees(swap_amount_rune, previous_state['rune_pool'], previous_state['asset_pool'], params['asset_chain_tx_fees'])    
    
def lp_fees_to_rune(swap_amount_asset, params, previous_state):
    return lp_fees(swap_amount_asset, previous_state['asset_pool'], 
                   previous_state['rune_pool'], params['thor_chain_tx_fees'])        

In [49]:
def p_add_lp(params, substep, state_history, previous_state):
    asset_threshold = to_asset(params['lp_threshold'], params)
    rune_threshold = to_rune(params['lp_threshold'], params)
    if (previous_state['asset_vault'] > asset_threshold + params['asset_chain_tx_fees']) and \
        (previous_state['rune_vault'] > rune_threshold + params['thor_chain_tx_fees']):
        return {
            'rune_fees': 0,
            'asset_chain_fees': params['asset_chain_tx_fees'],
            'thor_chain_fees': params['thor_chain_tx_fees'],
            'treasury_fees': 0,
            'lp_fees': 0,
            'asset_vault_change': -(asset_threshold + params['asset_chain_tx_fees']),
            'rune_vault_change': -(rune_threshold + params['thor_chain_tx_fees']),
            'asset_pool_change': asset_threshold,
            'rune_pool_change': rune_threshold,
            'asset_stake_change': asset_threshold,
            'rune_stake_change': rune_threshold
        }
    else:
        return {}

def p_swap_to_rune(params, substep, state_history, previous_state):
    swap_amount_asset = to_asset(random.rand() * params['max_swap_amount'], params)
    amount_rune = swap_to_rune(swap_amount_asset, previous_state)
    if swap_amount_asset > previous_state['asset_pool']:
        return {
            'rune_fees': 0,
            'asset_chain_fees': 0,
            'treasury_fees': 0,
            'lp_fees': 0,
            'asset_pool_change': 0,
            'rune_pool_change': 0,
        }
    
    return {
        'rune_fees': params['thor_chain_tx_fees'],
        'asset_chain_fees': params['asset_chain_tx_fees'],
        'treasury_fees': params['thor_chain_tx_fees'],
        'lp_fees': from_rune(lp_fees_to_rune(swap_amount_asset, params, previous_state), params),
        'asset_pool_change': -swap_amount_asset,
        'rune_pool_change': amount_rune
    }

def p_swap_to_asset(params, substep, state_history, previous_state):
    swap_amount_rune = to_rune(random.rand() * params['max_swap_amount'], params)
    amount_asset = swap_to_asset(swap_amount_rune, previous_state)
    if swap_amount_rune > previous_state['rune_pool']:
        return {
            'rune_fees': 0,
            'asset_chain_fees': 0,
            'treasury_fees': 0,
            'lp_fees': 0,
            'asset_pool_change': 0,
            'rune_pool_change': 0,
        }
    
    return {
        'rune_fees': params['thor_chain_tx_fees'],
        'asset_chain_fees': params['asset_chain_tx_fees'],
        'treasury_fees': params['asset_chain_tx_fees'],
        'lp_fees': from_asset(lp_fees_to_asset(swap_amount_rune, params, previous_state), params),
        'asset_pool_change': amount_asset,
        'rune_pool_change': -swap_amount_rune
    }

def p_remove_lp(params, substep, state_history, previous_state):
    
    
    return {
        'rune_fees': 0,
        'asset_chain_fees': 0,
        'treasury_fees': 0,
        'lp_fees': 0,
        'asset_pool_change': 0,
        'rune_pool_change': 0,
        'asset_vault_change': 0,
        'rune_vault_change': 0,
        'asset_stake_change': 0,
        'rune_stake_change': 0,
    }

def p_ilp_bonus(params, substep, state_history, previous_state):
    #TODO. Only applicable when removing.
    return {
        'rune_pool_change': 0,
    }

def p_add_to_vault(params, substep, state_history, previous_state):
    new_asset_deposit = 0
    new_rune_deposit = 0
    asset_staker_changes = [0] * len(previous_state['asset_stakers'])
    rune_staker_changes = [0] * len(previous_state['rune_stakers'])
        
    if (previous_state['timestep'] % 10 == 0):
        new_asset_deposit = to_asset(params['asset_stake_amount'], params)
        asset_staker = int(previous_state['timestep'] / 10)
        asset_staker_changes[asset_staker] = -new_asset_deposit

    if (previous_state['timestep'] % 5 == 0):
        new_rune_deposit = to_rune(params['rune_stake_amount'], params)
        rune_staker = int(previous_state['timestep'] / 10)
        rune_staker_changes[rune_staker] = -new_rune_deposit
    
    
    return {
        'asset_vault_change': new_asset_deposit,
        'rune_vault_change': new_rune_deposit,
        'asset_staker_changes': asset_staker_changes,
        'rune_staker_changes': rune_staker_changes
    }

In [50]:
def s_asset_vault(params, substep, state_history, previous_state, policy_input):
    return 'asset_vault', previous_state['asset_vault'] + policy_input['asset_vault_change']

def s_rune_vault(params, substep, state_history, previous_state, policy_input):    
    return 'rune_vault', previous_state['rune_vault'] + policy_input['rune_vault_change']

def s_asset_pool(params, substep, state_history, previous_state, policy_input):
    return 'asset_pool', previous_state['asset_pool'] + policy_input['asset_pool_change']

def s_rune_pool(params, substep, state_history, previous_state, policy_input):
    return 'rune_pool', previous_state['rune_pool'] + policy_input['rune_pool_change']

def s_rune_stake(params, substep, state_history, previous_state, policy_input):
    prev_stake_value = previous_state['rune_stake'] * previous_state['rune_pool']
    new_pool_value = policy_input['rune_stake_change'] + previous_state['rune_pool']
    if new_pool_value == 0.0:
        return 'rune_stake', 0.0

    rune_stake = (prev_stake_value + policy_input['rune_stake_change']) / new_pool_value
    return 'rune_stake', rune_stake

def s_asset_stake(params, substep, state_history, previous_state, policy_input):
    prev_stake_value = previous_state['asset_stake'] * previous_state['asset_pool']
    new_pool_value = policy_input['asset_stake_change'] + previous_state['asset_pool']
    if new_pool_value == 0.0:
        return 'asset_stake', 0.0
    rune_stake = (prev_stake_value + policy_input['asset_stake_change']) / new_pool_value
    return 'asset_stake', rune_stake

def s_asset_stakers(params, substep, state_history, previous_state, policy_input):
    asset_stakers = previous_state['asset_stakers']
    staker_changes = policy_input['asset_staker_changes']
    
    return 'asset_stakers', [{
        'asset': asset_stakers['asset'][i] + staker_changes[i],
        'asset_staked': -staker_changes[i],
        'asset_lp_ratio': 0 } for i in range(len(previous_state['asset_stakers']))]

def s_rune_stakers(params, substep, state_history, previous_state, policy_input):
    rune_stakers = previous_state['rune_stakers']
    staker_changes = policy_input['rune_staker_changes']    
    return 'rune_stakers', [{
        'rune': rune_stakers['rune'][i] + staker_changes[i],
        'rune_staked': -staker_changes[i],
        'rune_lp_ratio': 0 } for i in range(len(previous_state['rune_stakers']))]


In [51]:
state_update_blocks = [
    {
        'policies': {
            'add_lp': p_add_lp,
            'remove_lp': p_remove_lp,
            'add_to_vault': p_add_to_vault,
            # does this belong here given that it flows from remove_lp
#             'ilp_bonus': p_ilp_bonus,
        },
        'variables': {
            'asset_vault': s_asset_vault,
            'rune_vault': s_rune_vault,
            'asset_pool': s_asset_pool,            
            'rune_pool': s_rune_pool,
            'asset_stake': s_asset_stake,
            'rune_stake': s_rune_stake,
            'asset_stakers': s_asset_stakers,
            'rune_stakers': s_rune_stakers
        }
    },
    {
        'policies': {
            'swap_to_rune': p_swap_to_rune,
            'swap_to_asset': p_swap_to_asset,
        },
        'variables': {
            'asset_pool': s_asset_pool,
            'rune_pool': s_rune_pool,            
        }
    },
    
]

In [52]:
model = Model(
    # Model initial state
    initial_state=initial_state,
    # Model Partial State Update Blocks
    state_update_blocks=state_update_blocks,
    # System Parameters
    params=params
)

simulation = Simulation(
    model=model,
    timesteps=20,  # Number of timesteps
    runs=1  # Number of Monte Carlo Runs
)

# Executes the simulation, and returns the raw results
result = simulation.run()

df = pd.DataFrame(result)

Traceback (most recent call last):
  File "/Users/nishanths/opt/anaconda3/lib/python3.9/site-packages/radcad/core.py", line 100, in single_run
    _single_run(
  File "/Users/nishanths/opt/anaconda3/lib/python3.9/site-packages/radcad/core.py", line 76, in _single_run
    substate.update(updated_state)
  File "/Users/nishanths/opt/anaconda3/lib/python3.9/site-packages/radcad/core.py", line 13, in _update_state
    state_key, state_value = function(
  File "/var/folders/9t/pyh6d_sx4zj9dytr6955qcz80000gn/T/ipykernel_68532/3849670812.py", line 34, in s_asset_stakers
    return 'asset_stakers', [{
  File "/var/folders/9t/pyh6d_sx4zj9dytr6955qcz80000gn/T/ipykernel_68532/3849670812.py", line 35, in <listcomp>
    'asset': asset_stakers[i] + staker_changes[i],
TypeError: unsupported operand type(s) for +: 'dict' and 'float'



TypeError: unsupported operand type(s) for +: 'dict' and 'float'

In [ ]:
df